In [7]:
import sqlite3
import json
import cv2
import numpy as np
import base64
from keras.models import load_model

model1 = load_model('stage1-epoch-150.h5')
model2 = load_model('stage2-epoch-150.h5')
pix2pix = load_model('pix2pix-epoch-149.h5')

In [12]:
image = cv2.imread('../output/train/00000020.jpg', 1)
sketch = image[:,:256,:]
landmark = image[:,256:256*2,:]
image = image[:,256*2:,:]

_, sketch = cv2.threshold(sketch,200,255,cv2.THRESH_BINARY)

cv2.imwrite('./paper/image.png', image)
cv2.imwrite('./paper/sketch.png', sketch)
cv2.imwrite('./paper/landmark.png', landmark)

landmark = np.array(landmark)/127.5-1
sketch = np.array(sketch)/127.5-1

landmark = landmark[np.newaxis,...]
sketch = sketch[np.newaxis,...]

pix2pix_result = pix2pix.predict(sketch)
pix2pix_result = np.squeeze((pix2pix_result * 127.5 + 127.5).astype(np.uint8))
pix2pix_result = cv2.cvtColor(pix2pix_result, cv2.COLOR_RGB2BGR)
cv2.imwrite('./paper/pix2pix_result.png', pix2pix_result)

fake_sketch = model1.predict(landmark)
cv2.imwrite('./paper/fake_sketch.png', np.squeeze((fake_sketch * 127.5 + 127.5).astype(np.uint8)))

sketch = np.concatenate([fake_sketch, sketch], axis=3)
result = model2.predict(sketch)

result = np.squeeze((result * 127.5 + 127.5).astype(np.uint8))
result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

cv2.imwrite('./paper/result.png', result)

True